In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/car-body-damage/damage_car/Berat/IMG_9566JPG.jpg
/kaggle/input/car-body-damage/damage_car/Berat/IMG_4895.jpg
/kaggle/input/car-body-damage/damage_car/Berat/mobil108.jpg
/kaggle/input/car-body-damage/damage_car/Berat/_482.jpg
/kaggle/input/car-body-damage/damage_car/Berat/xsac.jpg
/kaggle/input/car-body-damage/damage_car/Berat/IMG_2174JPG - Copy.jpg
/kaggle/input/car-body-damage/damage_car/Berat/IMG_4603.jpg
/kaggle/input/car-body-damage/damage_car/Berat/mb-1.jpg
/kaggle/input/car-body-damage/damage_car/Berat/_0637.jpg
/kaggle/input/car-body-damage/damage_car/Berat/IMG_2038JPG.jpg
/kaggle/input/car-body-damage/damage_car/Berat/mbx.jpg
/kaggle/input/car-body-damage/damage_car/Berat/IMG_7201JPG.jpg
/kaggle/input/car-body-damage/damage_car/Berat/Bocah-SD-Ditabrak3.jpg
/kaggle/input/car-body-damage/damage_car/Berat/103717568-close-up-view-of-broken-car-crashed-in-front.jpg
/kaggle/input/car-body-damage/damage_car/Berat/mobil058.jpg
/kaggle/input/car-body-damage/damage_car/Bera

# Import

In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

from platform import python_version
print(python_version())

import warnings
import time
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image                  

import pandas as pd
import numpy as np
import seaborn as sn

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import tqdm
import random

import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix

import shutil
%matplotlib inline

from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

print(tf.__version__)

mem = psutil.virtual_memory()
print("mem", mem.total/1024/1024)

%system nvidia-smi
#%system rocm-smi

# Variables

In [ ]:
epochs = 100 
batch_size = 64 #150
testsplit = .2
targetx = 224
targety = 224
learning_rate = 0.0001
classes = 3 #jumlah kelas
seed = random.randint(1, 1000)

# Crop images using provided annotations

In [ ]:
data_dir = "/kaggle/input/bodymobil/body_mobil/gambar/"
annotations_dir = "/kaggle/input/bodymobil/body_mobil/annotation/"
cropped_dir = "/kaggle/input/car-body-damage/damage_car/"

%system rm -rf $cropped_dir
%system mkdir $cropped_dir

#this function adapted from https://www.kaggle.com/hengzheng/dog-breeds-classifier
def save_cropped_img(path, annotation, newpath):
    tree = ET.parse(annotation)
    xmin = int(tree.getroot().findall('.//xmin')[0].text)
    xmax = int(tree.getroot().findall('.//xmax')[0].text)
    ymin = int(tree.getroot().findall('.//ymin')[0].text)
    ymax = int(tree.getroot().findall('.//ymax')[0].text)
    image = Image.open(path)
    image = image.crop((xmin, ymin, xmax, ymax))
    image = image.convert('RGB')
    image.save(newpath)

breeds = os.listdir(data_dir)
annotations = os.listdir(annotations_dir)

print('breeds: ', len(breeds), 'annotations: ', len(annotations))

total_images = 0

for breed in breeds:
    dir_list = os.listdir(data_dir + breed)
    annotations_dir_list = os.listdir(annotations_dir + breed)
    img_list = [data_dir + breed + '/' + i for i in dir_list]
    os.makedirs(cropped_dir + breed)

    for file in img_list:
        annotation_path = annotations_dir + breed + '/' + os.path.basename(file[:-4])
        newpath = cropped_dir + breed + '/' + os.path.basename(file)
        save_cropped_img(file, annotation_path, newpath)
        total_images += 1
    
print("total images cropped", total_images)

# Keras image data readers

In [ ]:
cropped_dir = "/kaggle/input/car-body-damage/damage_car/"

In [ ]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=testsplit
)

train_generator = datagen.flow_from_directory(
        cropped_dir,
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=seed,
        subset="training"
)

test_generator = datagen.flow_from_directory(
        cropped_dir,
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed,
        subset="validation"
)

# Sample image

In [ ]:
img = train_generator.filepaths[np.random.random_integers(low=0, high=train_generator.samples)]
print(img)
img = mpimg.imread(img)
plt.imshow(img)

# Keras callbacks

In [ ]:
checkpoint = ModelCheckpoint('dog_breed_classifier.h5',
                             monitor='val_acc',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False)

#https://github.com/keras-team/keras/issues/3358
tensorboard = TensorBoard(log_dir="./logs-"+dt.datetime.now().strftime("%m%d%Y%H%M%S"),
                            histogram_freq=0,
                            batch_size=batch_size,
                            write_graph=False,
                            update_freq='epoch')

def epoch_end(epoch, logs):
    message = "End of epoch "+str(epoch)+". Learning rate: "+str(K.eval(model.optimizer.lr))
    os.system('echo '+message)

def epoch_begin(epoch, logs):
    print("Learning rate: ", K.eval(model.optimizer.lr))
    
def train_begin(logs):
    os.system("echo Beginning training")

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=.0001,
                          patience=20,
                          verbose=1,
                          mode='auto',
                          baseline=None,
                          restore_best_weights=True)

reducelr = ReduceLROnPlateau(monitor='val_loss',
                             factor=np.sqrt(.1),
                             patience=5,
                             verbose=1,
                             mode='auto',
                             min_delta=.0001,
                             cooldown=0,
                             min_lr=0.0000001)

lambdacb = LambdaCallback(on_epoch_begin=epoch_begin,
                          on_epoch_end=epoch_end,
                          on_batch_begin=None,
                          on_batch_end=None,
                          on_train_begin=train_begin,
                          on_train_end=None)

# Define new top layers and compile model

In [ ]:
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(rate = .2)(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu',  kernel_initializer='random_uniform', bias_initializer='zeros')(x)
# x = Dropout(rate = .2)(x)
x = BatchNormalization()(x)
predictions = Dense(classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(lr=learning_rate)
# optimizer = RMSprop(lr=learning_rate)

loss = "categorical_crossentropy"
# loss = "mean_squared_error"

for layer in model.layers:
    layer.trainable = True
# for layer in model.layers[-5:]:
#     layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

model.summary()
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

# Fit model

In [ ]:
%%time

params = model.fit_generator(generator=train_generator, 
                                steps_per_epoch=len(train_generator), 
                                validation_data=test_generator, 
                                validation_steps=len(test_generator),
                                epochs=epochs,
                                callbacks=[reducelr, earlystop, lambdacb, tensorboard, checkpoint])

# Training and test loss/accuracy graphs

In [ ]:
plt.subplot(1, 2, 1)
plt.title('Training and test accuracy')
plt.plot(params.epoch, params.history['accuracy'], label='Training accuracy')
plt.plot(params.epoch, params.history['val_accuracy'], label='Test accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.title('Training and test loss')
plt.plot(params.epoch, params.history['loss'], label='Training loss')
plt.plot(params.epoch, params.history['val_loss'], label='Test loss')
plt.legend()

plt.show()

# Sample prediction

In [ ]:
# Randomly test an image from the test set

# model.load_weights('dog_breed_classifier.h5')

imageno=np.random.random_integers(low=0, high=test_generator.samples)

name = test_generator.filepaths[imageno]
print(name)
plt.imshow(mpimg.imread(name))

img = Image.open(test_generator.filepaths[imageno]).resize((targetx, targety))
probabilities = model.predict(preprocess_input(np.expand_dims(img, axis=0)))
breed_list = tuple(zip(test_generator.class_indices.values(), test_generator.class_indices.keys()))

for i in probabilities[0].argsort()[-5:][::-1]: 
    print(probabilities[0][i], "  :  " , breed_list[i])

# Classification report

In [ ]:
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=len(test_generator))
y = np.argmax(predictions, axis=1)

print('Classification Report')
cr = classification_report(y_true=test_generator.classes, y_pred=y, target_names=test_generator.class_indices)
print(cr)

#akurasi 80 >